In [1]:
import numpy as np
import pySurrogate as pySur

**note**: This notebook currently has no regression testing. a more limited set of unit tests do exist, however.

In [2]:
# SaveH5Object was created to save the various classes used as hdf5 files.
# This notebook was created in case someone needs to make another subclass
# of SaveH5Object, although that seems unlikely.
SaveH5Object = pySur.SaveH5Object
print SaveH5Object.__doc__

A base class which Enables hierarchical saving and loading of data.


In [3]:
# SaveH5Object instances can be created, but usually we want to create another class using this as a base class.
# The cannonical example is to have some object which olds data as well as other objects.
class Holder(SaveH5Object):
    def __init__(self, name, data, subHolders):
        SaveH5Object.__init__(self, name)
        self.data = data
        self.subHolders = subHolders

        # We need to include some information on how to save this object - let's implement that a little later
        self.setupSaveH5Stuff()

    def iterH5Subordinates(self):
        # Anything yielded will be saved as an hdf5 group, with group name = item.name
        for holder in self.subHolders:
            yield holder

    def setupSaveH5Stuff(self):
        # We want to make sure we don't try to create an hdf5 data set out of an object!
        # Since the holders in self.subHolders will already be saved as a group, we can skip saving them as datasets
        # Any attribute key in self._doNotWriteToH5 will not be written as a dataset.
        self._doNotWriteToH5.append('subHolders')

        # When we load this object from a .h5 file, it first loads the datasets.
        # Then, it runs self._prepareForSubload()
        # Then, it tries to load all objects in self.__iter__ by using their name for the h5 group.
        # Since we don't have a fixed way of naming our subHolders, we need to store their names.
        self.subNames = [h.name for h in self.subHolders]

    def _prepareForSubload(self, isFirst):
        # This is called AFTER this object's data has been loaded but BEFORE the subHolders have been loaded.
        # isFirst will only be true for top-level SaveH5Objects, in case that is relevant.
        # We need to make sure that self.__iter__ yields the right kind of objects with the right names.
        # Let's just initialize empty holders with the right names, and their contents will then be loaded.
        self.subHolders = [Holder(name, None, []) for name in self.subNames]

In [4]:
# Make some holders which contain no subHolders
h1 = Holder('holder 1', 3, [])
h2 = Holder('holder 2', 'String data', [])
h3 = Holder('holder 3', np.arange(5), [])

# Make a holder containing some subHolders
group_holder1 = Holder('group 1', None, [h1, h2])
group_holder2 = Holder('group 2', {'Dicts' : 'can', 'be': 'saved', 'too': '!'}, [h3])

# Make a super holder containing the groups
main_holder = Holder('main holder', [0, 0, 5], [group_holder1, group_holder2])

In [5]:
# We can take a look at our objects:
print str(main_holder)
print '\n'
print repr(main_holder)

SaveH5Object: main holder


[(SaveH5Object: main holder): {'subHolders': [[(SaveH5Object: group 1): {'subHolders': [[(SaveH5Object: holder 1): {'subHolders': [], 'subNames': [], 'data': 3, 'name': 'holder 1', '_doNotWriteToH5': ['subHolders']}], [(SaveH5Object: holder 2): {'subHolders': [], 'subNames': [], 'data': 'String data', 'name': 'holder 2', '_doNotWriteToH5': ['subHolders']}]], 'subNames': ['holder 1', 'holder 2'], 'data': None, 'name': 'group 1', '_doNotWriteToH5': ['subHolders']}], [(SaveH5Object: group 2): {'subHolders': [[(SaveH5Object: holder 3): {'subHolders': [], 'subNames': [], 'data': array([0, 1, 2, 3, 4]), 'name': 'holder 3', '_doNotWriteToH5': ['subHolders']}]], 'subNames': ['holder 3'], 'data': {'be': 'saved', 'too': '!', 'Dicts': 'can'}, 'name': 'group 2', '_doNotWriteToH5': ['subHolders']}]], 'subNames': ['group 1', 'group 2'], 'data': [0, 0, 5], 'name': 'main holder', '_doNotWriteToH5': ['subHolders']}]


In [6]:
# Let's save the main_holder as a directory 'main_holder/'

# Remove it if needed:
import os
if os.path.exists('main_holder'):
    import shutil
    shutil.rmtree('main_holder')

main_holder.save('main_holder')

# Try looking at main_holder/info.txt and/or main_holder/data.h5

In [7]:
# Now let's create a brand new Holder and load the directory 'main_holder/'
new_holder = Holder('dummy name', None, [])
new_holder.load('main_holder')

# We end up with an exact copy of main_holder!
print repr(main_holder)

[(SaveH5Object: main holder): {'subHolders': [[(SaveH5Object: group 1): {'subHolders': [[(SaveH5Object: holder 1): {'subHolders': [], 'subNames': [], 'data': 3, 'name': 'holder 1', '_doNotWriteToH5': ['subHolders']}], [(SaveH5Object: holder 2): {'subHolders': [], 'subNames': [], 'data': 'String data', 'name': 'holder 2', '_doNotWriteToH5': ['subHolders']}]], 'subNames': ['holder 1', 'holder 2'], 'data': None, 'name': 'group 1', '_doNotWriteToH5': ['subHolders']}], [(SaveH5Object: group 2): {'subHolders': [[(SaveH5Object: holder 3): {'subHolders': [], 'subNames': [], 'data': array([0, 1, 2, 3, 4]), 'name': 'holder 3', '_doNotWriteToH5': ['subHolders']}]], 'subNames': ['holder 3'], 'data': {'be': 'saved', 'too': '!', 'Dicts': 'can'}, 'name': 'group 2', '_doNotWriteToH5': ['subHolders']}]], 'subNames': ['group 1', 'group 2'], 'data': [0, 0, 5], 'name': 'main holder', '_doNotWriteToH5': ['subHolders']}]


In [8]:
# cleanup
import os
if os.path.exists('main_holder'):
    import shutil
    shutil.rmtree('main_holder')
